In [ ]:
from google.colab import drive

drive.mount("/content/gdrive")

import pandas as pd
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

import os
from pathlib import Path

current_path = Path(os.getcwd())
# print(current_path)

data_path = 'gdrive/MyDrive/FOSCAR_2021/2021 국토부/yolo/data/'

data_file = os.listdir(data_path)
# print(data_file)

json_ext = r".json"
json_dt = [_ for _ in data_file if _.endswith(json_ext)]
# print(json_data)

import json

for j in range (len(json_dt)):
  json_data = json.load(open(data_path + json_dt[j]))
  print(json.dumps(json_data, indent="\t"))

  #이미지 출력하기
  df = pd.DataFrame(json_data["image"])
  img_info = json_data["image"]
  img_info

  img_data = img_info['filename']
  img_data
  img = cv2.imread(data_path +img_data, cv2.IMREAD_COLOR)
  # cv2_imshow(img)


  df = pd.DataFrame(json_data["annotation"])

  if len(df) != 0:
    df_traffic_light = df[df['class']=="traffic_light"]
    df_traffic_light = df[df['type']=="car"]

    # 인덱스 초기화
    df_traffic_light.reset_index(drop=True, inplace=True)
    df_traffic_light

    if len(df_traffic_light)!=0:
      # left_arrow -> left 로 바꾸기
      for i in range (len(df_traffic_light)):
        df_traffic_light['attribute'][i][0]['left'] = df_traffic_light['attribute'][i][0]['left_arrow']
        del df_traffic_light['attribute'][i][0]['left_arrow']

      for key,value in df_traffic_light['attribute'][0][0].items():
        if value =='on':
          # print(key)
          color = key

      color=[]
      for i in range (len(df_traffic_light['attribute'])):
        string=""
        for key,value in df_traffic_light['attribute'][i][0].items():
          if value =='on':
            # print(key)
            string += key
        # print(i,",",string)
        color.append(string)

      # 클래스를 name열로 추가하기
      df_traffic_light['name'] = ""

      # 신호등 정보(구수, 켜진 정보) 시각화
      for i in range (len(df_traffic_light)):
        img_result = cv2.rectangle(img,(df_traffic_light['box'][i][0],df_traffic_light['box'][i][1]),(df_traffic_light['box'][i][2],df_traffic_light['box'][i][3]),(0,255,0),2)
        img_result = cv2.putText(img_result,str(df_traffic_light['light_count'][i])+" "+color[i], (df_traffic_light['box'][i][0],df_traffic_light['box'][i][1]-15),0,0.8,(0,0,255),2)
        name = str(df_traffic_light['light_count'][i])+" "+color[i]
        df_traffic_light['name'][i]=name
        if name =='3 redleft':
          name = '3 left'
      cv2_imshow(img_result)

      # 'name'에 others_arrow있는 데이터 아예 삭제하기
      for i in range(len(df_traffic_light)):
        if 'others_arrow' in df_traffic_light['name'][i]:
          df_traffic_light = df_traffic_light.drop(i)
      # 인덱스 초기화
      df_traffic_light.reset_index(drop=True, inplace=True)
      df_traffic_light

      # 'name'에 x_light있는 데이터 아예 삭제하기
      for i in range(len(df_traffic_light)):
        if 'x_light' in df_traffic_light['name'][i]:
          df_traffic_light = df_traffic_light.drop(i)
      # 인덱스 초기화
      df_traffic_light.reset_index(drop=True, inplace=True)
      df_traffic_light

      # 3구 ,4구 아닌 신호등 삭제
      for i in range(len(df_traffic_light)):
        if df_traffic_light['light_count'][i]!='3' and df_traffic_light['light_count'][i]!='4':
          df_traffic_light = df_traffic_light.drop(i)
      # 인덱스 초기화
      df_traffic_light.reset_index(drop=True, inplace=True)
      df_traffic_light

      # 다 off인 데이터 삭제
      for k in range(len(df_traffic_light)):
        cnt=0
        the_values = df_traffic_light['attribute'][k][0].values()
        attribute = list(the_values)[0:6]
        for t in range (len(attribute)):
          if attribute[t]=='off':
            cnt+=1
            if (cnt==6):
              df_traffic_light = df_traffic_light.drop(k)
      # 인덱스 초기화
      df_traffic_light.reset_index(drop=True, inplace=True)
      df_traffic_light

    ############################################################################################################3
    ## 표지판!
    df_traffic_sign = df[df["class"]=="traffic_sign"]
    df_traffic_sign
    # 표지판 인덱스 초기화
    df_traffic_sign.reset_index(drop=True, inplace=True)
    df_traffic_sign

    if len(df_traffic_sign)!=0:
        # 표지판 시각화
        for i in range (len(df_traffic_sign)):
          img_rec2 = cv2.rectangle(img,(df_traffic_sign['box'][i][0],df_traffic_sign['box'][i][1]),(df_traffic_sign['box'][i][2],df_traffic_sign['box'][i][3]),(255,0,0),2)
        cv2_imshow(img_rec2)

        # 표지판 클래스 기입할 곳!
        df_traffic_sign['name'] = ""
        df_traffic_sign

        # 표지판 중 속도 제한 표지판 삭제하기
        for i in range(len(df_traffic_sign)):
          if ('text' in df_traffic_sign.keys()):
            if df_traffic_sign['text'][i]!='0':  #np.nan(df_traffic_sign['text'][i]) or   
              # ind = df_traffic_sign['text'][i].index
              df_traffic_sign = df_traffic_sign.drop(i)

  # 신호등이랑 표지판 데이터프레임 합치기
  result = pd.concat([df_traffic_light,df_traffic_sign], ignore_index=True)
  result

  # json파일로 저장
  result.to_json(data_path + json_dt[j],orient='records')

  # json파일 읽어오기
  json2 = pd.read_json(data_path + json_dt[j], orient='records')

  json_data2 = json.load(open(data_path + json_dt[j]))
  # print(json.dumps(json_data2, indent="\t") )
  json_data2

  final ={}
  final["annotation"] = json_data2
  final["image"]=json_data["image"]
  final

  # None인 것 제거하기
  for i in range (len(final["annotation"])):
    final["annotation"][i] = {key: value for key, value in final["annotation"][i].items() if value != None}
  final

  #딕셔너리 json으로 저장하기
  with open(data_path + json_dt[j], 'w') as outfile:
      json.dump(final, outfile)

Output hidden; open in https://colab.research.google.com to view.